In [ ]:
from keras import backend, layers, models, utils
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

# 분산 방식 Modeling - OOP

In [ ]:
def Conv2D(filters, kernel_size, padding='same', activation='relu'):
    return layers.Conv2D(filters, kernel_size, padding=padding, activation=activation)

class AE_Model_OOP(models.Model):
    def __init__(self, org_shape=(1, 28, 28)):
        original = layers.Input(shape=org_shape)

        x = Conv2D(4, (3, 3))(original)
        x = layers.MaxPooling2D((2, 2), padding='same')(x)

        x = Conv2D(8, (3, 3))(x)
        x = layers.MaxPooling2D((2, 2), padding='same')(x)

        z = Conv2D(1, (7, 7))(x)

        y = Conv2D(16, (3, 3))(z)
        y = layers.UpSampling2D((2, 2))(y)

        y = Conv2D(8, (3, 3))(y)
        y = layers.UpSampling2D((2, 2))(y)

        y = Conv2D(4, (3, 3))(y)

        decoded = Conv2D(1, (3, 3), activation='sigmoid')(y)

        super().__init__(original, decoded)

        self.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

# Data - MNIST

In [ ]:
class Data_MNIST():
    def __init__(self):
        num_classes = 10

        (x_train, y_train), (x_test, y_test) = mnist.load_data()
        img_rows, img_cols = x_train.shape[1:]

        if backend.image_data_format() == 'channels_first':
            x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
            x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
            input_shape = (1, img_rows, img_cols)
        else:
            x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
            x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
            input_shape = (img_rows, img_cols, 1)

        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train /= 255
        x_test /= 255

        y_train = utils.to_categorical(y_train, num_classes)
        y_test = utils.to_categorical(y_test, num_classes)

        self.input_shape = input_shape
        self.num_classes = num_classes
        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test

# Training and Evaluation

In [ ]:
def show_ae(autoencoder, data):
    decoded_imgs = autoencoder.predict(data.x_test)
    if backend.image_data_format() == 'channels_first':
        n, n_ch, n_i, n_j = data.x_test.shape
    else:
        n, n_i, n_j, n_ch = data.x_test.shape

    data.x_test = data.x_test.reshape(n, n_i, n_j)
    decoded_imgs = decoded_imgs.reshape(decoded_imgs.shape[0], n_i, n_j)

    n = 10
    plt.figure(figsize=(20, 4))
    for i in range(n):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(data.x_test[i], cmap='gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(decoded_imgs[i], cmap='gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

# Draw Result

In [ ]:
def plot_loss(history, title=None):
    if not isinstance(history, dict):
        history = history.history

    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    if title is not None:
        plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Training', 'Validation'], loc=0)
    plt.show()

def plot_acc(history, title=None):
    if not isinstance(history, dict):
        history = history.history

    plt.plot(history['accuracy'])
    plt.plot(history['val_accuracy'])
    if title is not None:
        plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Training', 'Validation'], loc=0)
    plt.show()

# Usage

In [ ]:
EPOCHS = 20
BATCH_SIZE = 128

data = Data_MNIST()
autoencoder = AE_Model_OOP(data.input_shape)
history = autoencoder.fit(data.x_train, data.x_train, epochs=EPOCHS, batch_size=BATCH_SIZE, shuffle=True, validation_split=0.2)

plot_acc(history)
plot_loss(history)
show_ae(autoencoder, data)